##### NB: This is Strictly for visualization of the data

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
DATA_FRAME=pd.read_csv("../data/yield_df.csv")

In [ ]:
DATA_FRAME.describe().T

In [ ]:
DATA_FRAME.drop("Unnamed: 0", axis=1,inplace=True)
DATA_FRAME

In [ ]:
# remove countries with less than 100 record
country_counts =DATA_FRAME['Area'].value_counts()
countries_to_drop = country_counts[country_counts < 100].index.tolist()
df_filtered = DATA_FRAME[~DATA_FRAME['Area'].isin(countries_to_drop)]
DATA_FRAME = df_filtered.reset_index(drop=True)

In [ ]:
COPY_DF=DATA_FRAME.copy()


categorical_columns = COPY_DF.select_dtypes(include=['object']).columns.tolist()
label_encoder = LabelEncoder()
for column in categorical_columns:
    COPY_DF[column] = label_encoder.fit_transform(COPY_DF[column])

COPY_DF

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

In [ ]:
X, y = COPY_DF.drop(labels='hg/ha_yield', axis=1), COPY_DF['hg/ha_yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
results = []

models = [
    ('Linear Regression', LinearRegression()),
    ('Random Forest', RandomForestRegressor(random_state=42)),
    ('Gradient Boost', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3,random_state=42)),
    ('XGBoost', XGBRegressor(random_state=42)),
    ('KNN',KNeighborsRegressor(n_neighbors=5)),
    ('Decision Tree',DecisionTreeRegressor(random_state=42)),
    ('Bagging Regressor',BaggingRegressor(n_estimators=150, random_state=42))
]

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test, y_test)
    MSE = mean_squared_error(y_test, y_pred)
    R2_score = r2_score(y_test, y_pred)
    results.append((name, accuracy, MSE, R2_score))
    acc = (model.score(X_train , y_train)*100)
    print(f'The accuracy of the {name} Model Train is {acc:.2f}')
    acc =(model.score(X_test , y_test)*100)
    print(f'The accuracy of the  {name} Model Test is {acc:.2f}')
    print("----------------------------------------------------------------------")

dff = pd.DataFrame(results, columns=['Model', 'Accuracy', 'MSE', 'R2_score'])
df_styled_best = dff.style.highlight_max(subset=['Accuracy','R2_score'], color='green').highlight_min(subset=['MSE'], color='green').highlight_max(subset=['MSE'], color='red').highlight_min(subset=['Accuracy','R2_score'], color='red')

display(df_styled_best)


In [ ]:
#Creating persistent model:
import joblib

# Learning with the Model
X, y = COPY_DF.drop(labels='hg/ha_yield', axis=1), COPY_DF['hg/ha_yield']


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(model, 'standard-scaler.joblib')



model= BaggingRegressor(n_estimators=150, random_state=42)
model.fit(X.values, y)

#Create a Persisting Model
joblib.dump(model, 'crop-predictor.joblib')



In [ ]:
import pandas as pd
import joblib
# 	Area	Item	Year	hg/ha_yield	average_rain_fall_mm_per_year	pesticides_tonnes	avg_temp

# Testing created model:

scaler = joblib.load('standard-scaler.joblib')

scaled_inputs = [[100,5,2013,657.0,2550.07,19.76]]

model=joblib.load('crop-predictor.joblib')
predictions = model.predict(scaled_inputs)

print("Prediction:",predictions)